In [1]:
import json
from pprint import pprint

In [2]:
with open('train.json') as f:
    data1=json.load(f)

data1.keys()
#pprint(data)

with open('test.json') as f:
    data2=json.load(f)

data1.keys()
#pprint(data)

dict_keys(['index', 'content', 'date', 'tags', 'title', 'url'])

In [3]:
import pandas as pd
set=[]
for k,v in data1['content'].items():
    #print(k)
    #print(v)
    set.append(v)

for k,v in data2['content'].items():
    #print(k)
    #print(v)
    set.append(v)
    
df1=pd.DataFrame(set)
print(df1.head())
print(df1.shape)    
    
df1=pd.DataFrame(set)
df1.head()
df1.shape

                                                   0
0  Honda is recalling the 2017 Honda Ridgeline pi...
1  This morning, Honda issued two safety recalls ...
2  2016 was dominated by some pretty major headli...
3  Last week, Hyundai recalled some 5,600 crossov...
4  Fiat Chrysler Automobiles is recalling nearly ...
(2835, 1)


(2835, 1)

In [4]:
df1.columns=['a']

In [5]:
df1['a']

0       Honda is recalling the 2017 Honda Ridgeline pi...
1       This morning, Honda issued two safety recalls ...
2       2016 was dominated by some pretty major headli...
3       Last week, Hyundai recalled some 5,600 crossov...
4       Fiat Chrysler Automobiles is recalling nearly ...
5       Hyundai is recalling more than 5,600 Tucson an...
6       The massive, record-breaking recall of Takatas...
7       The National Highway Traffic Safety Administra...
8       A little over a year ago, AutoNation made a ve...
9       Ford has issued two recalls affecting nearly 6...
10      Nissan is recalling select 2016 Titan Diesel X...
11      Toyota is recalling the 2011-2016 Toyota Sienn...
12      Kia has announced a recall of the 2008 and 200...
13      This morning, Fiat Chrysler Automobiles issued...
14      Mazda is recalling RX-8 vehicles from the 2004...
15      Mitsubishi is recalling roughly 195,000 U.S. v...
16      Volvo has issued recalls for 2016 and 2017 mod...
17      Kia is

In [14]:
#nltk.download()

In [6]:

import nltk
from nltk.corpus import stopwords
mystopwords=stopwords.words("English") + ['one', 'become', 'get', 'make', 'take', 'recall','said', 'say', 'could', 'nhtsa', 'n\'t', 'may', 'vehicle', 'car']
WNlemma = nltk.WordNetLemmatizer()

def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    text_after_process=" ".join(tokens)
    return(text_after_process)


In [7]:
toks_train = df1['a'].apply(pre_process)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [9]:
# Create tfidf matrix
vectorizer = TfidfVectorizer(max_features=2500,
                              stop_words=mystopwords,
                             use_idf=True)
X = vectorizer.fit_transform(toks_train)
X.shape

(2835, 2500)

In [10]:
# Use SVD to reduce dimensions
svd = TruncatedSVD(500)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_lsa = lsa.fit_transform(X)


In [11]:
# Check how much variance is explained
explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

Explained variance of the SVD step: 76%


In [12]:
# Now the actual clustering
from sklearn.cluster import KMeans
#random_state=4321
km3 = KMeans(n_clusters=5, init='k-means++', max_iter=1000, n_init=1)
%time km3.fit(X_lsa)

#‘k-means++’ : selects initial cluster centers for k-mean clustering in a smart way to speed up convergence.
#n_init : int, default: 10. Number of time the k-means algorithm will be run with different centroid seeds.
#Maximum number of iterations of the k-means algorithm for a single run.

Wall time: 169 ms


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
    n_clusters=5, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [13]:
from sklearn import metrics



# Silhouette: more similar within clusters, more distant between clusters
# The higher the better (-1 to 1)

print("Silhouette Coefficient for 3 clusters: %0.3f"
      % metrics.silhouette_score(X_lsa, km3.labels_))

Silhouette Coefficient for 3 clusters: 0.033


In [14]:

def print_terms(cm, num):
    original_space_centroids = svd.inverse_transform(cm.cluster_centers_)
    order_centroids = original_space_centroids.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(num):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

print_terms(km3, 5)

Cluster 0: blogsmith widget fivemin image display 2015 none postcontentcontainer cke_show_borders block
Cluster 1: ford owner safety model chrysler brake 2014 steering seat affected
Cluster 2: fuel tank ford pump leak engine fire owner model safety
Cluster 3: cars com find dealer owner service local department free repair
Cluster 4: toyota takata million honda inflator company airbag automaker lexus safety
